In [45]:
import requests
import time
import random
import re
from openpyxl import Workbook
from bs4 import BeautifulSoup
from urllib.request import Request
from urllib.request import urlopen

UAS = ("Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) Gecko/20100101 Firefox/40.1", 
       "Mozilla/5.0 (Windows NT 6.3; rv:36.0) Gecko/20100101 Firefox/36.0",
       "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10; rv:33.0) Gecko/20100101 Firefox/33.0",
       "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36",
       "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.1 Safari/537.36",
       "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.0 Safari/537.36",
       )

ua = UAS[random.randrange(len(UAS))]
headers = {'user-agent': ua}

In [28]:
req = Request(url= 'https://www.skinstore.com/blog/skincare/page/2/', headers = headers)
html = urlopen(req).read()
soup = BeautifulSoup(html, features='lxml')
b_list = []
for link in soup.find_all('h4',class_='post-card__title'):
    url= link.find('a')
    b_list.append(url['href'])
print(b_list)
len(b_list)

['https://www.skinstore.com/blog/events/brilliant-gift-ideas-for-mothers-day/', 'https://www.skinstore.com/blog/skincare/how-to-get-your-fix-with-nuface/', 'https://www.skinstore.com/blog/skincare/which-vitamin-c-serum-is-right-for-me/', 'https://www.skinstore.com/blog/body/this-just-in-our-newest-brands-this-month/', 'https://www.skinstore.com/blog/skincare/all-about-our-latest-brand-addition-rodial/', 'https://www.skinstore.com/blog/skincare/feeling-22-whats-in-our-anniversary-giveaway-bag/', 'https://www.skinstore.com/blog/skincare/glow-to-bed-glycolic-10-renew-overnight/', 'https://www.skinstore.com/blog/skincare/your-winter-skin-problems-solved-with-la-roche-posay/', 'https://www.skinstore.com/blog/skincare/farm-to-beauty-juice-beautys-full-cycle-mission-to-ensure-all-natural-ingredients/', 'https://www.skinstore.com/blog/skincare/female-founders-why-we-say-yes-to-sunday-riley/']


10

In [46]:
def get_urls(url):
    """从每个列表页获取文章url"""
    req = Request(url= url, headers = headers)
    html = urlopen(req).read()
    soup = BeautifulSoup(html, features='lxml')
    b_list = []
    for link in soup.find_all('h4',class_='post-card__title'):
        url= link.find('a')
        b_list.append(url['href'])
    return b_list

In [47]:
len(get_urls('https://www.skinstore.com/blog/skincare/page/2/'))

10

In [48]:
def get_info(url):
    """获取标题、作者、发布时间及正文内容"""
    time.sleep(0.1)
    req = Request(url, headers = headers)
    html = urlopen(req).read()
    soup = BeautifulSoup(html, features='lxml')
  
    if soup.find('h1', class_='post-header__title') == None:
        return {'url': url, 'title': None, 'author and date': None, 'body': None}
    else:
        title = soup.find('h1', class_='post-header__title').text
        author_date = soup.find('a',class_='post-meta__link').text
        p_list=[]
        for p in soup.find_all('div', class_='post-content-inner'):
            p_list.append(p.text)
        body = '\n'.join(p_list)
        return {'url': url, 'title': title, 'author and date': author_date, 'body': body}

In [49]:
def save_as_xls(info_list):
    """将获取到的文章信息存储到本地"""
    workbook = Workbook()
    worksheet = workbook.active
    worksheet.title = 'sheet1'
    titles = ['URL', 'Title', 'Author and Date', 'Content']
    keys = ['url', 'title', 'author and date', 'body']
    for index, title in enumerate(titles, start=1):
        worksheet.cell(row=1, column=index, value=title)
    for i, info in enumerate(info_list, start=1):
        for j,key in enumerate(keys, start=1):
            worksheet.cell(row=i+1, column=j, value=info[key])
    workbook.save('info.xlsx')

In [50]:
list=['https://www.skinstore.com/blog/skincare/']
for i in range(2,53):
    list.append('https://www.skinstore.com/blog/skincare/page/'+ str(i))
len(list)

52

In [51]:
def main():
    """main function"""
    info_list = []
    for i in list:
        b_list = get_urls(i)
        for url in b_list:
            info_list.append(get_info(url))
    save_as_xls(info_list)
    
if __name__ == "__main__":
    main()